# LegislationAnalytics: Парсинг и анализ данных из системы обеспечения законодательной деятельности

Этот файл содержит код, реализующий этапы сбора и обработки данных, необходимых для анализа факторов, влияющих на скорость принятия законопроектов. Основное внимание уделено автоматизированному парсингу данных с сайта "Система обеспечения законодательной деятельности" и структурированию информации для дальнейшего анализа.

### Парсинг страниц законопроектов

In [2]:
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from itertools import chain
import warnings
warnings.filterwarnings("ignore")

Напишем функцию, которая забирает все ссылки на законопроекты с одной страницы результатов:

In [3]:
def get_all_links(link):
    """
    Return the list of all links from 
    one page of results (10 by default)
    """
    page = requests.get(link)
    soup = BeautifulSoup(page.text)

    divs = soup.find_all("div", class_ = "o_top")
    links = [d.find("a") for d in divs]
    links_clean = list(filter(lambda x : x is not None, links))
    hrefs = [a.get("href") for a in links_clean]
    
    return hrefs

Теперь можем подставлять вместо `i` в `page` любое число в цикле и получать списки со ссылками. 

In [4]:
conv6 = []

for i in range(1, 717):
    li = f"https://sozd.duma.gov.ru/oz?b[Convocation][]=6&b[ClassOfTheObjectLawmakingId]=1&page={i}&#data_source_tab_b"
    res = get_all_links(li)
    conv6.extend(res)
    sleep(3)

In [7]:
conv7 = []
for i in range(1, 574):
    li = f"https://sozd.duma.gov.ru/oz?b[Convocation][]=7&b[ClassOfTheObjectLawmakingId]=1&page={i}&#data_source_tab_b"
    res = get_all_links(li)
    conv7.extend(res)
    sleep(3)

Сформируем список ссылок на страницы законопроектов, представляющий собой последовательность URL-адресов в полном формате. Это упростит последующий процесс парсинга информации с каждой страницы. Для сохранения промежуточных результатов и предотвращения потери данных экспортируем эти ссылки в файл, что обеспечит их доступность для повторного использования.

In [14]:
final_6 = ["https://sozd.duma.gov.ru" + li for li in conv6]
df_conv_6 = pd.DataFrame(final_6)
df_conv_6.to_csv("conv6.csv")

final_7 = ["https://sozd.duma.gov.ru" + li for li in conv7]
df_conv_7 = pd.DataFrame(final_7)
df_conv_7.to_csv("conv7.csv")

### Извлечение данных со страниц законопроектов

После парсинга HTML-контента выполняется обработка и извлечение ключевой информации о законопроектах. Используется библиотека BeautifulSoup для поиска и структурирования данных.

Напишем функцию, которая забирает всю нужную нам информацию со страницы законопроекта:

In [16]:
def get_bill(link0, convocation):
    try:
        # Отправляем запрос на страницу
        page = requests.get(link0)
        soup = BeautifulSoup(page.text)

        # Парсинг основной таблицы
        table = soup.find("table")
        if table is None:
            raise ValueError("Таблица с данными не найдена на странице.")
        df = pd.read_html(str(table))[0].set_index(0).T

        # Извлечение номера законопроекта и ссылки
        title = soup.find("title").text.split()[0]
        df["Номер"] = title
        df["Ссылка"] = link0

        # Парсинг дат
        date_init = soup.find_all("div", class_="bh_etap_date")[0].find("span", class_="mob_not").text
        date_acept = soup.find_all("div", class_="bh_etap_date")[-1].find("span", class_="mob_not").text
        df["Дата внесения"] = date_init
        df["Дата принятия/не принятия"] = date_acept

        # Определение статуса законопроекта
        h_bar = soup.find("div", class_="bill_gorizontal_progress")
        if h_bar is None:
            raise ValueError("Полоса прогресса законопроекта не найдена.")
        divs = h_bar.find_all("div")
        classes = [d.get("class") for d in divs]
        classes_plain = list(chain.from_iterable(classes))
        greens_c = classes_plain.count("green")
        status = "принят" if greens_c == 8 else "не принят"
        df["Статус"] = status

        # Добавляем номер созыва
        df["Созыв"] = convocation

        return df

    except requests.exceptions.RequestException as e:
        print(f"Ошибка сети при обработке ссылки {link0}: {e}")
    except ValueError as e:
        print(f"Ошибка данных при обработке ссылки {link0}: {e}")
    except Exception as e:
        print(f"Неизвестная ошибка при обработке ссылки {link0}: {e}")

    return None

Напишем функцию, которая будет циклом парсить данные со страниц:

In [19]:
def parse_bills(links, convocation):
    parsed_bills = []
    for link in links:
        try:
            bill_data = get_bill(link, convocation=convocation)
            if bill_data is not None:  
                parsed_bills.append(bill_data)
        except Exception as e:
            print(f"Ошибка при обработке ссылки {link}: {e}")
    return parsed_bills

Запустим функцию для каждого созыва, объеденим датафреймы и выгрузим в `csv` файл.

In [22]:
all_bills = []

bills_6 = parse_bills(final_6, convocation=6)

Ошибка данных при обработке ссылки https://sozd.duma.gov.ru/bill/1098517-6: Таблица с данными не найдена на странице.
Неизвестная ошибка при обработке ссылки https://sozd.duma.gov.ru/bill/481943-6: 'NoneType' object has no attribute 'text'


In [27]:
bills_6

[0 Субъект права законодательной инициативы Форма законопроекта  \
 1  Депутат Государственной Думы Д.В.Ушаков   Федеральный закон   
 
 0                              Ответственный комитет  \
 1  Комитет Государственной Думы по труду, социаль...   
 
 0                Отрасль законодательства Тематический блок законопроектов  \
 1  060.000.000 Труд и занятость населения              Социальная политика   
 
 0                                 Профильный комитет  \
 1  Комитет Государственной Думы по труду, социаль...   
 
 0 Пакет документов при внесении       Номер  \
 1                           NaN  №1187320-6   
 
 0                                   Ссылка Дата внесения  \
 1  https://sozd.duma.gov.ru/bill/1187320-6    04.10.2016   
 
 0 Дата принятия/не принятия     Статус  Созыв  
 1                10.03.2017  не принят      6  ,
 0 Субъект права законодательной инициативы Форма законопроекта  \
 1  Депутат Государственной Думы Д.В.Ушаков   Федеральный закон   
 
 0             

In [28]:
bills_7 = parse_bills(final_7, convocation=7)

In [ ]:
all_bills = []

bills_6 = parse_bills(final_6, convocation=6)
bills_7 = parse_bills(final_7, convocation=7)
all_bills = pd.concat(bills_6 + bills_7, ignore_index=True)
all_bills.to_csv("all_bills_6th_and_7th_convocation.csv", index=False)

In [31]:
all_bills = pd.concat(bills_6 + bills_7, ignore_index=True)
all_bills.to_csv("all_bills_6th_and_7th_convocation.csv", index=False)

In [ ]:
# на входе: ссылка на один з/п
# на выходе: датафрейм + CSV для одного з/п

def get_bill(link0):
    page = requests.get(link0)
    soup = BeautifulSoup(page.text)
    df = pd.read_html(str(soup.find("table")))[0].set_index(0).T
    title = soup.find("title").text.split()[0]
    df["Номер"] = title
    df["Ссылка"] = link0
    
    date_introd = soup.find_all("div", 
                                class_ = "bh_etap_date")[0].find("span", 
                                                                 class_ = "mob_not").text
    date_fin = soup.find_all("div", 
      class_ = "bh_etap_date")[-1].find("span", class_ = "mob_not").text
    df["Дата внесения"] = date_introd
    df["Дата принятия/не принятия"] = date_fin
    
    h_bar = soup.find("div", class_ = "bill_gorizontal_progress")
    divs = h_bar.find_all("div")
    classes = [d.get("class") for d in divs]
    classes_plain = list(chain.from_iterable(classes))
    greens_c = classes_plain.count("green")
    
    if greens_c == 8:
        status = "принят"
    else:
        status = "не принят"
    df["Статус"] = status
    df.to_csv(title + ".csv")
    return df

In [ ]:
dat_links = pd.read_excel("conv6_part01.xlsx")
to_iterate = dat_links[0].values

for i in to_iterate:
    get_bill(i)
    sleep(2)
    print(i) # на случай, если остановится/сломается, видеть ссылку